In [1]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import shapely.geometry as sg
import matplotlib

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

DATA_DIR = DIR/'data/'

%load_ext autoreload
%autoreload 2
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# List feed

path = DATA_DIR/'cairns_gtfs.zip'
gt.list_gtfs(path)

In [ ]:
# Read and print feed

feed = gt.read_gtfs(path, dist_units='km')
print(feed)

In [ ]:
feed.describe()

In [ ]:
# Validate

feed.validate()

In [ ]:
# Compute trip stats

trip_stats = feed.compute_trip_stats()
trip_stats.head().T

In [ ]:
# Add shape_dist_traveled column to stop times

feed = feed.append_dist_to_stop_times(trip_stats)
feed.stop_times.head().T

In [ ]:
# Choose study dates

week = feed.get_first_week()
dates = [week[4], week[6]]  # First Friday and Sunday
dates

In [ ]:
# Compute feed time series

ts = feed.compute_feed_time_series(trip_stats, dates, freq='6H')
ts

In [ ]:
gt.downsample(ts, freq='12H')


In [ ]:
# Compute feed stats for first week

feed_stats = feed.compute_feed_stats(trip_stats, week)
feed_stats

In [ ]:
import plotnine as p9

(
    p9.ggplot(feed_stats, p9.aes(x='date', y='service_duration', group=1))
    + p9.geom_line(colour="blue", linetype="dashed")
    + p9.geom_point(colour="blue", size=4, shape='o', fill="white")
    + p9.xlab("Date") 
    + p9.ylab("Service duration (h)")
    + p9.ggtitle("First Monday--Sunday week of feed")
)

In [ ]:
# Compute route time series

rts = feed.compute_route_time_series(trip_stats, dates, freq='6H')
rts.head()

In [ ]:
# Slice time series

inds = ['service_distance', 'service_duration', 'service_speed']
rids = ['110-423', '110N-423']

rts.loc[:, (inds, rids)]

In [ ]:
# Compute trip locations for every hour

rng = pd.date_range('1/1/2000', periods=24, freq='H')
times = [t.strftime('%H:%M:%S') for t in rng]
loc = feed.locate_trips(dates[0], times)
loc.head()

In [ ]:
# Build a route timetable

route_id = feed.routes['route_id'].iat[0]
feed.build_route_timetable(route_id, dates).T

In [ ]:
# Compute screen line counts

path = DATA_DIR/'cairns_screen_line.geojson'
with path.open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])
print(line)

feed.compute_screen_line_counts(line, dates)

In [ ]:
# You need to install Folium for this to work

rids = feed.routes.route_id.loc[2:4]
feed.map_routes(rids)
